In [1]:
#install 
!sudo apt install tesseract-ocr
!pip install easyocr
!pip install opencv-python-headless==4.1.2.30
!pip install pdf2image
!apt-get install poppler-utils 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 62 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 0s (9,826 kB/s)
debconf: unable to initi

In [30]:
import easyocr
import numpy as np
from pdf2image import convert_from_path
import cv2
from PIL import Image
import os
from google.colab.patches import cv2_imshow

In [40]:
path='/content/sample_data/UntitledFolder'
fil=1
fil1=1
reader = easyocr.Reader(['en']) 
for i in os.listdir(path):
    print(i)
    print(i[-3:])
    if(i[-3:]=='PDF' or i[-3:]=='pdf'):
        images = convert_from_path(path+'/'+i)
        print(len(images))
        line=''
        for j in images:
            j.save(str(fil1)+'.png')
            image1 = cv2.imread(str(fil1)+'.png')
            fil1+=1
            gray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
            thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
        
            # Remove horizontal lines
            horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (50,1))
            detect_horizontal = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)
            cnts = cv2.findContours(detect_horizontal, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cnts = cnts[0] if len(cnts) == 2 else cnts[1]
            for c in cnts:
                cv2.drawContours(thresh, [c], -1, (0,0,0), 2)
            #cv2_imshow(thresh)
            # Dilate to connect text and remove dots
            kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (12,1))
            dilate = cv2.dilate(thresh, kernel, iterations=4)
            #cv2_imshow(dilate)
            cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cnts = cnts[0] if len(cnts) == 2 else cnts[1]
            for c in cnts:
                area = cv2.contourArea(c)
                if area < 200:
                    cv2.drawContours(dilate, [c], -1, (0,0,0), -1)

            result = cv2.bitwise_and(image1, image1, mask=dilate)
            result[dilate==0] = (255,255,255)
            #cv2_imshow(result)
            result1 = reader.readtext(result,detail=0)
            #print(result1)
            for res in result1:
                line=line+' '+res
        print()
        print(line)
        print()
    else:
        image = cv2.imread(path+'/'+i)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

        # Remove horizontal lines
        horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (50,1))
        detect_horizontal = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)
        cnts = cv2.findContours(detect_horizontal, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = cnts[0] if len(cnts) == 2 else cnts[1]
        for c in cnts:
            cv2.drawContours(thresh, [c], -1, (0,0,0), 2)
        #cv2_imshow(thresh)
        # Dilate to connect text and remove dots
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (10,1))
        dilate = cv2.dilate(thresh, kernel, iterations=1)
        #cv2_imshow(dilate)
        cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = cnts[0] if len(cnts) == 2 else cnts[1]
        for c in cnts:
            area = cv2.contourArea(c)
            if area < 200:
                cv2.drawContours(dilate, [c], -1, (255,255,255), -1)

        result = cv2.bitwise_and(image, image, mask=dilate)
        result[dilate==0] = (255,255,255)
        #cv2_imshow(result)
        result1 = reader.readtext(result,detail=0,paragraph=True)
        #print(result1)
        line=''
        for res in result1:
            line=line+' '+res
        print()
        print(line)
        print()
    '''f= open('/content/sample_data/UntitledFolder1/'+str(fil)+".txt","w+")
    for i in line:
        f.write(i)
    f.close()
    print('/content/sample_data/UntitledFolder1/'+str(fil)+".txt")
    fil+=1'''

financial_document_form 16_2019_11_26-12_44_23_597570_.pdf
pdf
6

 TDS TRACES Centralized Processing Cell TDS Reconciliation Analysis and Correction Enabling System Governmen: of India Income Tax Department FORM NO. 16 [See rule 31(1)(a)] PART A Certificate under Section 203 of the Income-tax Act; 1961 for tax deducted at source 0n salary Certificate No.  VKMPMKC Last updated on 16-Jul-2019 Name and address of the Employer Name and address of the Employee INVESTORS CLINIC INFRATECH PRIVATE LIMITED N-5, OPP_ AXIS BANK, SECTOR-18, NEERAJ SHARMA NOIDA 201301 B 4 BASMENT, SAXENA HOUSE, SHIVALIK NEAR, MALVIYA Uttar Pradesh NAGAR, DELHI 110017 Delhi +(91)120-4618200 FINANCEICIPL@GMAILCOM Employee Reference No PAN of the Employee PAN of the Deductor TAN of the Deductor provided by the Employer (If available) AACCI2147R MRTI00933C ANZPS4822J CIT (TDS) Assessment Year Period with the Employer From To The Commissioner of Income Tax (TDS) 110/25-26, 80 ft road; Ashok nagar; Opp. Sant Paul School 

In [10]:
path='/content/sample_data/UntitledFolder1'
nlp = spacy.blank("en") # load a new spacy model
db = DocBin()
for i in os.listdir(path):
    if i[-4:]=='json':
        f = open(path+'/'+i)
        TRAIN_DATA = json.load(f)
        for text, annot in tqdm(TRAIN_DATA['annotations']): 
            doc = nlp.make_doc(text) 
            ents = []
            for start, end, label in annot["entities"]:
                span = doc.char_span(start, end, label=label, alignment_mode="contract")
                if span is None:
                    print("Skipping entity")
                else:
                    ents.append(span)
            doc.ents = ents 
            print(doc.ents)
            db.add(doc)
db.to_disk("val_small_data.spacy")

FileNotFoundError: ignored